In [5]:
import pandas as pd
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
import pickle
import datatable as dt
import scipy.spatial.distance as spd
import MC as MC

In [6]:
### beta data loading preporceessed data 
sarcoma = dt.fread('beta.csv')
sarcoma=sarcoma.to_pandas()


#### model loading
model = torch.load('model.pt') ### 고정된것   
model.to(device)
model.eval()
device = torch.device("cuda")
### features loading
features=pd.read_csv('Probe_scores.csv')  ### 고정된것   
features=list(features.iloc[:2000,0])
#### 만약에 feature랑 겹치는 애들 없으면 0으로 채우기 ## 가끔 이런 경우 있음
if len(set(features) -set(sarcoma.columns)) !=0:
    sarcoma[(set(features) -set(sarcoma.columns))]=0
xx= sarcoma.loc[:,features]

##### xx 가 Input Data. + active function 1 - tanh 0- relu
xx=torch.from_numpy(np.array(xx))
xx=xx.to(device)
z=model(xx.float(),1)
pred = torch.max(z.data,1)
pred = pred[1].to("cpu") ### prediction value
categories = list(range(0, 91))
#scores = z.reshape((30,1,91)).to("cpu")
scores = z.reshape((-1,1,91)).to("cpu")
distance='euclidean'



#### open max model loading
test_loss = 0
correct = 0
total = 0
weibull_alpha=3
weibull_tail=20
threshold=0.5

### 고정된것  
with open('model.pickle', 'rb') as f:
    weibull_model = pickle.load(f)

####     
import scipy.spatial.distance as spd
pred_softmax, pred_softmax_threshold, pred_openmax = [], [], []
score_softmax, score_openmax = [], []
for score in scores:
    so, ss = openmax(weibull_model, categories, score,
                     0.9, weibull_alpha,distance)  # openmax_prob, softmax_prob
    pred_softmax.append(np.argmax(ss))
    pred_softmax_threshold.append(np.argmax(ss) if np.max(ss) >= threshold else 91)
    pred_openmax.append(np.argmax(so) if np.max(so) >= threshold else 91)
    score_softmax.append(ss)
    score_openmax.append(so)
    
#sample - predicted_label - percentage -second predicted_label - percentage
# if 92 label just percentage na value


for score in scores:
    so, ss = openmax(weibull_model, categories, score,
                     0.9, weibull_alpha,distance)  # openmax_prob, softmax_prob
    pred_softmax.append(np.argmax(ss))
    pred_softmax_threshold.append(np.argmax(ss) if np.max(ss) >= threshold else 91)
    pred_openmax.append(np.argmax(so) if np.max(so) >= threshold else 91)
    score_softmax.append(ss)
    score_openmax.append(so)
    

label=pd.read_csv('label.csv') ### 고정된것   
t=list(label['0'])
t.append('un_defined')
score=pd.DataFrame(score_softmax)
score.columns=label['0']
score1=pd.DataFrame(score_openmax)
score1.columns=t

result=pd.DataFrame(index=sarcoma.index,columns=['predicted_label','first_score','second_predicted_label','second_score'])
for idx,i in enumerate(score.index):
    if score1.iloc[0,:].sort_values(ascending=False).index[0] =='un_defined':
        result.loc[i,'predicted_label']='un_defined'
        result.loc[i,'first_score']=score1.iloc[idx,:].sort_values(ascending=False)[0]
    else:
        result.loc[i,'predicted_label']=score1.iloc[idx,:].sort_values(ascending=False).index[0]
        result.loc[i,'first_score']=score1.iloc[idx,:].sort_values(ascending=False)[0]
    result.loc[i,'second_predicted_label']=score1.iloc[idx,:].sort_values(ascending=False).index[1]
    result.loc[i,'second_score']=score1.iloc[idx,:].sort_values(ascending=False)[1]

ValueError: File D:\jaeminjj\SNU\methylation_classifier\Untitled Folder\beta.csv does not exist

In [ ]:
pd.read_csv